## Construct the dataset

In [1]:
import requests
import re
import string
from bs4 import BeautifulSoup
import bs4
import pickle
import numpy as np
import keras

/Users/laurensuarez/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
with open("data/bible_data_20181123_update.pkl", "rb") as handle:
    bible_data=pickle.load(handle)
kjv_bible_data=bible_data["King James Bible"]

## Heatmap 

Of verse dependecies based on cross reference

## Mappings

In [4]:
int2verse={}
verse2int={}
for i,v in enumerate(kjv_bible_data.keys()):
    int2verse[i]=v
    verse2int[v]=i
int2verse
verse2int

{'Genesis 1:1': 0,
 'Genesis 1:2': 1,
 'Genesis 1:3': 2,
 'Genesis 1:4': 3,
 'Genesis 1:5': 4,
 'Genesis 1:6': 5,
 'Genesis 1:7': 6,
 'Genesis 1:8': 7,
 'Genesis 1:9': 8,
 'Genesis 1:10': 9,
 'Genesis 1:11': 10,
 'Genesis 1:12': 11,
 'Genesis 1:13': 12,
 'Genesis 1:14': 13,
 'Genesis 1:15': 14,
 'Genesis 1:16': 15,
 'Genesis 1:17': 16,
 'Genesis 1:18': 17,
 'Genesis 1:19': 18,
 'Genesis 1:20': 19,
 'Genesis 1:21': 20,
 'Genesis 1:22': 21,
 'Genesis 1:23': 22,
 'Genesis 1:24': 23,
 'Genesis 1:25': 24,
 'Genesis 1:26': 25,
 'Genesis 1:27': 26,
 'Genesis 1:28': 27,
 'Genesis 1:29': 28,
 'Genesis 1:30': 29,
 'Genesis 1:31': 30,
 'Genesis 2:1': 31,
 'Genesis 2:2': 32,
 'Genesis 2:3': 33,
 'Genesis 2:4': 34,
 'Genesis 2:5': 35,
 'Genesis 2:6': 36,
 'Genesis 2:7': 37,
 'Genesis 2:8': 38,
 'Genesis 2:9': 39,
 'Genesis 2:10': 40,
 'Genesis 2:11': 41,
 'Genesis 2:12': 42,
 'Genesis 2:13': 43,
 'Genesis 2:14': 44,
 'Genesis 2:15': 45,
 'Genesis 2:16': 46,
 'Genesis 2:17': 47,
 'Genesis 2:18': 48,

## Storing mappings

In [5]:
kjv_bible_mapping={}
num_verses=len(int2verse.keys())
i=0
for k,obj in kjv_bible_data.items():
    #print(k, obj[0], obj[1])
    mapping=np.zeros((num_verses))
    for cf in obj[1]:
        mapping[verse2int[cf]]=1
    kjv_bible_mapping[k]=[obj[0],mapping]
    i+=1
    
    if i % 1000 == 0:
        print(i)
kjv_bible_mapping["Genesis 1:1"][1]

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000


array([0., 0., 0., ..., 0., 0., 0.])

## Save mappings and dataset

In [9]:
class MacOSFile(object):

    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        # print("reading total_bytes=%s" % n, flush=True)
        if n >= (1 << 31):
            buffer = bytearray(n)
            idx = 0
            while idx < n:
                batch_size = min(n - idx, 1 << 31 - 1)
                # print("reading bytes [%s,%s)..." % (idx, idx + batch_size), end="", flush=True)
                buffer[idx:idx + batch_size] = self.f.read(batch_size)
                # print("done.", flush=True)
                idx += batch_size
            return buffer
        return self.f.read(n)

    def write(self, buffer):
        n = len(buffer)
        print("writing total_bytes=%s..." % n, flush=True)
        idx = 0
        while idx < n:
            batch_size = min(n - idx, 1 << 31 - 1)
            print("writing bytes [%s, %s)... " % (idx, idx + batch_size), end="", flush=True)
            self.f.write(buffer[idx:idx + batch_size])
            print("done.", flush=True)
            idx += batch_size

def pickle_dump(obj, file_path):
    with open(file_path, "wb") as f:
        return pickle.dump(obj, MacOSFile(f), protocol=pickle.HIGHEST_PROTOCOL)


def pickle_load(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(MacOSFile(f))

In [10]:
pickle_dump(kjv_bible_mapping, "data/kjv_bible_mapping.pkl")
pickle_dump(int2verse, "data/int2verse_mapping.pkl")
pickle_dump(verse2int, "data/verse2int_mapping.pkl")
print("Successful saving.")

writing total_bytes=7744824500...
writing bytes [0, 1073741824)... done.
writing bytes [1073741824, 2147483648)... done.
writing bytes [2147483648, 3221225472)... done.
writing bytes [3221225472, 4294967296)... done.
writing bytes [4294967296, 5368709120)... done.
writing bytes [5368709120, 6442450944)... done.
writing bytes [6442450944, 7516192768)... done.
writing bytes [7516192768, 7744824500)... done.
writing total_bytes=571892...
writing bytes [0, 571892)... done.
writing total_bytes=571892...
writing bytes [0, 571892)... done.
Successful saving.


In [5]:
print(kjv_bible_data["Genesis 1:1"])
int2verse[np.argmax(kjv_bible_mapping["Genesis 1:1"][1])]

['In the beginning God created the heaven and the earth.', ['John 1:1', 'John 1:2', 'Acts 17:24', 'Hebrews 1:10', 'Hebrews 11:3', 'Revelation 4:11', 'Nehemiah 9:6', 'Job 9:8', 'Job 38:4', 'Psalm 89:11', 'Psalm 102:25', 'Psalm 115:15', 'Psalm 124:8', 'Psalm 136:5', 'Psalm 148:5', 'Isaiah 40:21', 'Isaiah 42:5', 'Isaiah 45:18', 'Jeremiah 10:12', 'Jeremiah 51:15']]


'Nehemiah 9:6'

In [6]:
bible_graph = np.matrix(list(map(lambda k:k[1], kjv_bible_mapping.values())))
print(bible_graph.shape)
int2verse[np.argmax(bible_graph[0])] #first row came in correctly

(31102, 31102)


'Nehemiah 9:6'

In [ ]:
bible_graph.shape

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

def show_graph_with_labels(adjacency_matrix, mylabels):
    rows, cols = np.where(adjacency_matrix == 1)
    edges = zip(rows.tolist(), cols.tolist())
    gr = nx.Graph()
    gr.add_edges_from(edges)
    nx.draw(gr, node_size=500, labels=mylabels, with_labels=True)
    plt.show()

show_graph_with_labels(bible_graph, int2verse)

/Users/laurensuarez/anaconda3/lib/python3.6/site-packages/networkx/drawing/layout.py:499: RuntimeWarning: invalid value encountered in sqrt
  distance = np.sqrt((delta**2).sum(axis=0))


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

def show_graph_with_labels(adjacency_matrix, mylabels):
    rows, cols = np.where(adjacency_matrix == 1)
    edges = zip(rows.tolist(), cols.tolist())
    gr = nx.Graph()
    gr.add_edges_from(edges)
    nx.draw(gr, node_size=500, labels=mylabels, with_labels=True)
    plt.show()

show_graph_with_labels(bible_graph, int2verse.values())